<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/financial_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Stock Screening Inspired by IBD's EPS Rating

### Install and Setup (this section will be executed automatically)

#### Install Required Packages

In [ ]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ibd_fin"
%pip install requests-cache

!wget -O NotoSansTC-Regular.ttf https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ibd_fin) to /tmp/pip-req-build-2km_vm9s
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-2km_vm9s
  Running command git checkout -b feature/ibd_fin --track origin/feature/ibd_fin
  Switched to a new branch 'feature/ibd_fin'
  Branch 'feature/ibd_fin' set up to track remote branch 'feature/ibd_fin' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit 81ebd7aba5050ed8a5955c2459bf7c79a6751ff0
  Preparing metadata (setup.py) ... done
--2024-10-01 12:39:52--  https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0
Resolving share.cole.tw (share.cole.tw)... 180.218.0.193
Connecting to share.cole.tw (share.cole.tw)|180.218.0.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7110560 (6.8M) [application/octet-stream]
Saving to:

#### Setup and Configuration

In [ ]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [ ]:
# @title Set Chinese Font for matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fm.fontManager.addfont('NotoSansTC-Regular.ttf')
font_name = 'Noto Sans TC'
if font_name not in plt.rcParams['font.sans-serif']:
    plt.rcParams['font.sans-serif'].insert(0, font_name)

In [ ]:
# @title ReadOnlyGitHub
import requests
import pandas as pd
from io import StringIO

class ReadOnlyGitHub:
    def __init__(self, repo_owner, repo_name, dir='', branch='main'):
        dir = dir.strip('/')
        base = 'https://raw.githubusercontent.com'
        if dir:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}/{dir}'
        else:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}'

        base = 'https://api.github.com/repos'
        if dir:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents'

    def file_exists(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        response = requests.head(url)
        return response.status_code == 200

    def list_filenames(self, dir_path=''):
        url = f'{self.api_url}/{dir_path}'
        response = requests.get(url)
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
            return []

    def download_csv(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        if self.file_exists(file_path):
            return pd.read_csv(url)
        else:
            return pd.DataFrame()

# Create a GitHub instance
github = ReadOnlyGitHub(
    repo_owner='yorkjong',
    repo_name='stock-reports',
    dir='ibd_fin'
)

In [ ]:
# @title DataFrame Utilities

def print_column(df, column):
    if column in df.columns:
        print(', '.join(df[column]))

In [ ]:
# @title Source of Tickers

def tickers_from_df(df):
    if 'Name' in df.columns:
        return list(df['Name'])
    elif 'Ticker' in df.columns:
        return list(df['Ticker'])
    return []

def filter1_tickers():
    return list(tickers_from_df(df_top_f1))

def major_indices():
    return ['^DJI', '^IXIC', '^NDX', '^RUT', '^SOX',
            '^TWII', '^N225', '^HSI',
            '^STOXX50E', '^FTSE', '^GDAXI', '^FCHI', '^GSPTSE']

def sector_indices():
    return ['SOXX', 'DVY',
            'IWB','IWM', 'IWV',  'IJR',
            'ITB', 'IHI', 'IYC', 'ITA', 'IAK',
            'IYZ', 'IYT', 'IYR', 'IYF', 'IYJ',
            'IYG', 'IYH', 'IYK', 'IDU', 'IYE', 'IHE',
            'IAT', 'IAI', 'IEO', 'IYM', 'IHF']

In [ ]:
# @title Checkboxes

import ipywidgets as widgets

def cbs_create(symbols, n_pre_checked=10):
    '''Create a list of checkboxes'''
    return [
        widgets.Checkbox(
            value=(i < n_pre_checked),  # Set first n items as checked
            description=symbol,
            layout=widgets.Layout(width='auto'),
            style={'description_width': 'auto'}
        )
        for i, symbol in enumerate(symbols)
    ]

def cbs_with_grid(checkboxes, n_cols=5):
    '''Create a grid layouting the given checkboxes'''
    return widgets.GridBox(checkboxes, layout=widgets.Layout(
        width='auto',
        grid_template_columns=f'repeat({n_cols}, 1fr)',
        grid_gap='10px'  # Add some space between the checkboxes
    ))

def cbs_get_selected(checkboxes):
    '''Get the selected symbols from the given checkboxes'''
    return [checkbox.description for checkbox in checkboxes if checkbox.value]

def cbs_unselect_all(checkboxes):
    '''Unselect all checkboxes in the given list'''
    for checkbox in checkboxes:
        checkbox.value = False

def cbs_select_top(checkboxes, n=10):
    '''Select the top n checkboxes in the given list'''
    for i, checkbox in enumerate(checkboxes):
        checkbox.value = (i < n)

In [ ]:
# @title Dropdown Menus

def create_period_dropdown(value='2y'):
    return widgets.Dropdown(
        options=['1y', '2y', '5y'],
        value=value,
        description='Period:',
    )

def create_interval_dropdown(value='1wk'):
    return widgets.Dropdown(
        options=['1d', '1wk'],
        value=value,
        description='Interval:',
    )

def create_style_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['default', 'classic', 'yahoo', 'charles', 'tradingview', 'binance', 'binancedark', 'mike', 'nightclouds', 'checkers', 'ibd', 'sas', 'starsandstripes', 'kenan', 'blueskies', 'brasil'],
        value='yahoo' if value is None else value,
        description='Style:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

def create_template_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['plotly', 'plotly_white', 'plotly_dark', 'ggplot2', 'seaborn', 'simple_white', 'presentation', 'xgridoff', 'ygridoff'],
        value='plotly_dark' if value is None else value,
        description='Template:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

In [ ]:
# @title Multiple Searchable Dropdown Menus
import ipywidgets as widgets

def create_search_box():
    '''Create a Text widget for search input'''
    return widgets.Text(
        description='Search:',
        placeholder='Type to search',
        layout=widgets.Layout(width='auto')
    )

def create_dropdown(options, description='Stock:'):
    '''Create a Dropdown widget for displaying filtered options'''
    return widgets.Dropdown(
        description=description,
        options=[None] + options,  # None as the default option
        layout=widgets.Layout(width='auto'),
        value=None  # Set default value to None
    )

def update_dropdown(change, dropdown, options):
    '''Update the options in the dropdown based on search input'''
    search_text = change['new'].lower()
    filtered = [option for option in options if search_text in option.lower()]
    if filtered:
        dropdown.options = [None] + filtered
        dropdown.value = filtered[0]  # Auto-select the first matching option
    else:
        dropdown.options = [None]  # Retain only the None option if no match

def remove_duplicates_preserve_order(lst):
    '''Remove duplicates from a list while preserving order'''
    seen = set()
    result = []
    for item in lst:
        if item and item not in seen:
            seen.add(item)
            result.append(item)
    return result

def get_dropdowns_selected_options(dropdowns):
    '''Get selected options from the dropdowns'''
    selected = [dropdown.value for dropdown in dropdowns if dropdown.value]
    return remove_duplicates_preserve_order(selected)

def create_search_dropdowns(options, max_selections):
    '''Create a layout with search boxes and dropdowns'''
    # Create UI components
    search_boxes = [create_search_box() for _ in range(max_selections)]
    dropdowns = [create_dropdown(options) for _ in range(max_selections)]

    # Bind search box and dropdown menu events
    for search_box, dropdown in zip(search_boxes, dropdowns):
        search_box.observe(lambda change, dropdown=dropdown:
                            update_dropdown(change, dropdown, options),
                            names='value')
    # Create the layout
    controls = [widgets.HBox([search_box, dropdown])
                for search_box, dropdown  in zip(search_boxes, dropdowns)]
    layout = widgets.VBox(controls)
    return dropdowns, layout

In [ ]:
# @title Outputs
outputs = widgets.VBox()

In [ ]:
# @title enable_plotly_in_cell
# ref. https://stackoverflow.com/questions/76593068/plotly-figure-not-rendering-in-ipywidgets-interact-function-google-colab
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

### Glossary of Terms

#### Source (The Source of Stocks to Analyze)
- **Description**: Stocks can be sourced from various exchanges or indices.
- **Common Abbreviations**:
  - **Taiwan Markets**:
    - `TWSE`: Taiwan Stock Exchange (台灣上市股票交易所)
    - `TPEX`: Taipei Exchange (上櫃交易所)
    - `ESB`: Emerging Stock Board (興櫃交易所)
    - Combine with `+` (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - **America Markets**:
    - `SPX`: S&P 500 (標普五百指數)
    - `DJIA`: Dow Jones Industrial Average (道瓊指數)
    - `NDX`: NASDAQ-100 (納斯達克一百指數)
    - `RUI`: Russell 1000
    - `RUT`: Russell 2000
    - `SOX`: PHLX Semiconductor Index (費半指數)
    - `W5000`: Wilshire 5000 Total Market Index
    - `USLS`: U.S. Listed Stocks (美國上市股票)
    - `U.S.Listed`: U.S. Listed Stocks (美國上市股票)
    - Combine with `+` (e.g., `SPX+DJIA+NDX+SOX`)

#### Period (Historical Data Time Range)
- **Description**: The duration for fetching historical data.
- **Example**: `2y` for 2 years

#### Interval (Historical Data Frequency)
- **Description**: The frequency of historical data points.
- **Common Intervals**:
  - `'1d'`: Daily data
  - `'1wk'`: Weekly data
  - `'1mo'`: Monthly data

#### EPS RS (EPS Relative Strength)
- **Description**: Measures a stock's earnings per share (EPS) performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the EPS performance of the benchmark index.
  - A positive value indicates that the stock's EPS growth has outperformed the benchmark.
  - A negative value indicates underperformance relative to the benchmark.
- **Index EPS Calculation**: Weighted according to component stocks' share counts.

#### Rev RS (Operating Revenue Relative Strength)
- **Description**: Measures a stock's operating revenue performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the revenue performance of the benchmark index.
  - A positive value indicates that the stock's revenue growth has outperformed the benchmark.
  - A negative value indicates underperformance relative to the benchmark.
- **Index Revenue Calculation**: Weighted according to component stocks' market capitalization.

#### TTM EPS (Trailing Twelve Months EPS)
- **Description**: The earnings per share over the trailing twelve months, which gives a more recent view of the company's profitability.
- **Unit**: Currency per share
- **Interpretation**:
  - A higher TTM EPS indicates better profitability over the past year.
  - Used to compare with previous periods or with other companies.

#### TTM RPS (Trailing Twelve Months Revenue Per Share)
- **Description**: The revenue per share over the trailing twelve months, showing how much revenue a company generates per share.
- **Unit**: Currency per share
- **Interpretation**:
  - A higher TTM RPS indicates better revenue generation capability per share over the past year.
  - Useful for comparing revenue performance with previous periods or with other companies.

#### TTM PE (Trailing Twelve Months PE)
- **Description**: The price-to-earnings ratio calculated using the earnings from the trailing twelve months. It provides a measure of how much investors are willing to pay per dollar of earnings.
- **Unit**: Ratio
- **Interpretation**:
  - A higher TTM PE indicates that investors expect higher future growth and are willing to pay more for the stock.
  - A lower TTM PE might suggest undervaluation or lower expected growth.


## Execute Actions Step by Step

In [23]:
# @title Step 1. Pick a File

import ipywidgets as widgets

with requests_cache.disabled():
    filenames = github.list_filenames()

# Extract and sort all unique values in reverse order
all_dates = sorted(set(fn.split('_')[3].replace('.csv', '') for fn in filenames), reverse=True)
all_sources = sorted(set(fn.split('_')[0] for fn in filenames))
all_types = sorted(set(fn.split('_')[1] for fn in filenames), reverse=True)

# Calculate the maximum length of filenames to set dropdown width
max_filename_length = max(len(fn) for fn in filenames)
dropdown_width = f'{max_filename_length * 10}px'  # 10px width per character

# Create dropdowns with dynamic width
date_dropdown = widgets.Dropdown(
    options=all_dates,
    description='Date:',
    layout=widgets.Layout(width=dropdown_width)
)

source_dropdown = widgets.Dropdown(
    options=all_sources,
    description='Source:',
    layout=widgets.Layout(width=dropdown_width)
)

type_dropdown = widgets.Dropdown(
    options=all_types,
    description='Type:',
    layout=widgets.Layout(width=dropdown_width)
)

# Create file selection dropdown
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    layout=widgets.Layout(width=dropdown_width)
)

def update_dropdowns(*args):
    # Filter files based on selected date
    date_filtered_files = [fn for fn in filenames if date_dropdown.value in fn]

    # Update Source dropdown
    available_sources = sorted(set(fn.split('_')[0] for fn in date_filtered_files))
    source_dropdown.options = available_sources
    if source_dropdown.value not in available_sources:
        source_dropdown.value = available_sources[0] if available_sources else None

    # Update Type dropdown
    available_types = sorted(set(fn.split('_')[1] for fn in date_filtered_files), reverse=True)
    type_dropdown.options = available_types
    if type_dropdown.value not in available_types:
        type_dropdown.value = available_types[0] if available_types else None

    # Update file options
    update_file_options()

def update_file_options(*args):
    filtered_files = [
        fn for fn in filenames
        if (date_dropdown.value in fn and
            source_dropdown.value == fn.split('_')[0] and
            type_dropdown.value == fn.split('_')[1])
    ]
    file_dropdown.options = filtered_files
    if filtered_files:
        file_dropdown.value = filtered_files[0]  # Set initial value to the first match
    else:
        file_dropdown.value = None

# Bind event handlers
date_dropdown.observe(update_dropdowns, 'value')
source_dropdown.observe(update_file_options, 'value')
type_dropdown.observe(update_file_options, 'value')

# Display all dropdowns
display(date_dropdown, source_dropdown, type_dropdown, file_dropdown)

# Initialize dropdowns
update_dropdowns()

Dropdown(description='Date:', layout=Layout(width='340px'), options=('20241001',), value='20241001')

Dropdown(description='Source:', layout=Layout(width='340px'), options=('DJIA', 'NDX', 'RUI', 'SOX', 'SPX', 'TW…

Dropdown(description='Type:', layout=Layout(width='340px'), options=('stocks',), value='stocks')

Dropdown(description='File:', layout=Layout(width='340px'), options=(), value=None)

### Step 2. Filter Stocks

In [32]:
# @title Filter 1 {"run":"auto"}
cond0 = "((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))" # @param ["((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))", ""]
cond1 = "& (df['EPS RS (%)'] > 0)" # @param ["& (df['EPS RS (%)'] > 0)", "& (df['EPS RS (%)'] > 50)", "& (df['EPS RS (%)'] > 100)", "& (df['EPS RS (%)'] > 200)", "& (df['EPS RS (%)'] > 500)", ""]
cond2 = "& (df['Rev RS (%)'] > 0)" # @param ["& (df['Rev RS (%)'] > 0)", "& (df['Rev RS (%)'] > 5)", "& (df['Rev RS (%)'] > 10)", "& (df['Rev RS (%)'] > 20)", "& (df['Rev RS (%)'] > 50)", "& (df['Rev RS (%)'] > 100)", ""]
cond3 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond4 = "& (df['EPS YoY (%)'] > 0)" # @param ["& (df['EPS YoY (%)'] > 0)", "& ((df['EPS YoY (%)'] > 0) & (df['YoY 2Q Algo (%)'] > 0))", ""]
cond5 = "& ((df['EPS QoQ (%)'] > 0) & (df['QoQ 2Q Algo (%)'] > 0) & (df['QoQ 3Q Algo (%)'] > 0))" # @param ["& (df['EPS QoQ (%)'] > 0)", "& ((df['EPS QoQ (%)'] > 0) & (df['QoQ 2Q Algo (%)'] > 0))", "& ((df['EPS QoQ (%)'] > 0) & (df['QoQ 2Q Algo (%)'] > 0) & (df['QoQ 3Q Algo (%)'] > 0))", ""]
cond6 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
cond7 = "& (df['Sector'] == 'Technology')" # @param ["& (df['Sector'] == 'Technology')", "& (df['Sector'] == 'Energy')", ""]
sorted_column = "EPS RS (%)" # @param ["EPS RS (%)", "Rev RS (%)"]
ascending = False # @param {"type":"boolean"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 8
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

selected_file = file_dropdown.value
source, kind, fin, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nDate: {date}")

import numpy as np

df = github.download_csv(selected_file)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df[eval(cond)]
df = df.sort_values(by=sorted_column, ascending=ascending)
df_top_f1 = df.head(num_items)
display(df_top_f1)

print_column(df_top_f1, 'Name')
print_column(df_top_f1, 'Ticker')

Source: U.S.Listed
Type: stocks
Date: 20241001


,Ticker,Sector,Industry,Price,EPS QoQ (%),QoQ 2Q Algo (%),QoQ 3Q Algo (%),EPS YoY (%),YoY 2Q Algo (%),EPS RS (%),TTM EPS,Rev RS (%),TTM RPS,TTM PE,EPS RS Rank (%),RPS RS Rank (%)
14,NXT,Technology,Solar,38.90,0.12,0.14,0.05,0.95,37.25,22225787.74,3.78,31.85,27.133,9.92,99.70,87.73
269,VERX,Technology,Software—Application,36.73,0.50,2000000.00,2000000.00,2.67,7.00,996.20,0.13,4.39,4.021,296.23,94.32,71.23
420,PAY,Technology,Software—Infrastructure,20.01,0.33,0.09,0.10,0.60,5.00,599.13,0.26,19.38,5.646,77.00,91.13,83.38
492,PLTR,Technology,Software - Infrastructure,36.84,0.20,0.25,0.33,5.00,NaN,505.45,0.17,16.27,1.127,218.82,89.61,81.80
567,YOU,Technology,Software—Application,32.64,0.30,0.08,0.09,5.50,4.33,435.99,0.78,19.14,7.586,42.49,88.03,83.24
576,NVDA,Technology,Semiconductors,121.40,0.13,0.23,0.31,1.72,6.28,429.19,2.14,91.55,3.910,56.75,87.84,92.43
746,IMMR,Technology,Software—Application,8.95,0.52,0.22,5.12,3.14,NaN,334.98,1.80,872.92,5.119,4.96,84.25,95.54
773,APP,Technology,Software - Application,127.75,0.31,0.37,0.59,3.18,NaN,323.49,2.34,30.65,11.726,55.79,83.68,87.46
822,GB,Technology,Software—Infrastructure,5.30,0.25,0.33,23.32,1.50,NaN,305.77,0.13,46.06,1.894,42.08,82.64,89.75
1522,ODD,Technology,Software—Infrastructure,39.72,0.39,0.78,3.57,0.49,0.57,159.46,1.54,30.02,10.429,26.22,67.86,87.25


NXT, VERX, PAY, PLTR, YOU, NVDA, IMMR, APP, GB, ODD


### Step 3. Visualize Filtered Stocks

In [ ]:
# @title Plot 1. Mansfield Relative Strength Comparison {"run":"auto"}
source = "Filter 1" # @param ["Filter 1", "Major Global Stock Indices", "Sector Indices"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import matplotlib.pyplot as plt
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

MAX_STOCK_SELECTION = 10
N_COLS = 8  # the number of columns of the grid layout for checkboxes

symbols = {
    'Filter 1': filter1_tickers,
    'Major Global Stock Indices': major_indices,
    'Sector Indices': sector_indices,
}[source]()

checkboxes = cbs_create(symbols, MAX_STOCK_SELECTION)
checkbox_grid = cbs_with_grid(checkboxes, N_COLS)
btn_unselect_all = widgets.Button(description="Unselect All")
btn_unselect_all.on_click(lambda b: cbs_unselect_all(checkboxes))
btn_select_top = widgets.Button(description="Select Top 10")
btn_select_top.on_click(lambda b: cbs_select_top(checkboxes, MAX_STOCK_SELECTION))

period_dropdown = create_period_dropdown()
interval_dropown = create_interval_dropdown()

cmp_theme_dropdown = {
    'mplfinance': create_style_dropdown('Comparison Theme:', 'charles'),
    'Plotly': create_template_dropdown('Comparison Theme:', 'plotly_dark'),
}[backend]
btn_plot = widgets.Button(description="Generate Plot")

out_msg, out_fig = widgets.Output(), widgets.Output()

ui = widgets.VBox([
    checkbox_grid,
    widgets.HBox([btn_unselect_all, btn_select_top]),
    widgets.VBox([period_dropdown, interval_dropown]),
    widgets.VBox([cmp_theme_dropdown, btn_plot]),
    out_msg, out_fig
])
display(ui)

import vistock.mpl.mansfield as mpl_mansfield
import vistock.plotly.mansfield as ply_mansfield

rs_cmp = {
    'mplfinance': mpl_mansfield.RelativeStrengthLines,
    'Plotly': ply_mansfield.RelativeStrengthLines,
}[backend]

def on_checkbox_change(change):
    selected_count = sum([cb.value for cb in checkboxes])
    if selected_count > MAX_STOCK_SELECTION:
        # Uncheck the last checked box if selection exceeds limit
        changed_checkbox = change['owner']
        changed_checkbox.value = False
        with out_msg:
            out_fig.clear_output()
            print(f"Only {MAX_STOCK_SELECTION} stocks can be selected at most.")

# Bind the checkbox change event to the function
for checkbox in checkboxes:
    checkbox.observe(on_checkbox_change, names='value')

def on_plot_click(b):
    symbols = cbs_get_selected(checkboxes)
    if not symbols:
        with out_msg:
            out_fig.clear_output()
            print("No stocks selected. Please select at least one stock.")
        return
    with out_fig:
        out_msg.clear_output()
        clear_output()
        interval = interval_dropown.value
        period = period_dropdown.value
        if rs_cmp is mpl_mansfield.RelativeStrengthLines:
            rs_cmp.plot(symbols, interval=interval, period=period,
                        style=cmp_theme_dropdown.value,
                        color_cycle=plt.cm.Paired.colors)
        else: # Plotly
            rs_cmp.plot(symbols, interval=interval, period=period,
                        template=cmp_theme_dropdown.value,
                        colorway=px.colors.qualitative.Set3)

btn_plot.on_click(on_plot_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


In [ ]:
# @title Plot 2. Mansfield Stock Chart {"run":"auto"}
source = "Filter 1" # @param ["Filter 1","Major Global Stock Indices", "U.S. Listed Stocks", "Taiwan Stocks"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import functools as ft
import ipywidgets as widgets
from IPython.display import display, clear_output
import yfinance as yf

import vistock.stock_indices as si
import vistock.tw as tw

symbols = {
    'Filter 1': filter1_tickers,
    'Major Global Stock Indices': major_indices,
    'U.S. Listed Stocks': ft.partial(si.get_tickers, 'USLS'),
    'Taiwan Stocks': ft.partial(si.get_tickers, 'TWSE+TPEX+ESB'),
}[source]()
if source == 'Taiwan Stocks':
    symbols = [f"{tw.stock_name(s)} {s}" for s in symbols]

dropdowns, layout = create_search_dropdowns(symbols, 1)

period_dropdown2 = create_period_dropdown('2y')
interval_dropown2 = create_interval_dropdown('1wk')
stock_theme_dropdown = {
    'mplfinance': create_style_dropdown('Stock Theme:', 'yahoo'),
    'Plotly': create_template_dropdown('Stock Theme:', 'plotly'),
}[backend]

btn_plot_prc = widgets.Button(description="Price/RS/Volume Chart",
                              layout=widgets.Layout(width='168px'))
btn_plot_fin = widgets.Button(description="Financial Chart",
                              layout=widgets.Layout(width='168px'))
btn_report_q = widgets.Button(description="Quarterly Report",
                              layout=widgets.Layout(width='168px'))
btn_report_a = widgets.Button(description="Annual Report",
                              layout=widgets.Layout(width='168px'))
btn_clear_last = widgets.Button(description="Clear Last")
btn_clear_all = widgets.Button(description="Clear All")
#outputs = widgets.VBox()

ui = widgets.VBox([
    layout,
    period_dropdown2, interval_dropown2, stock_theme_dropdown,
    widgets.HBox([btn_plot_prc, btn_plot_fin]),
    widgets.HBox([btn_report_q, btn_report_a]),
    widgets.HBox([btn_clear_last, btn_clear_all]),
    outputs
])
display(ui)

import vistock.mpl.mansfield as mpl_mansfield
import vistock.plotly.mansfield as ply_mansfield

stock_chart = {
    'mplfinance': mpl_mansfield.StockChart,
    'Plotly': ply_mansfield.StockChart,
}[backend]

def get_symbols():
    symbols = get_dropdowns_selected_options(dropdowns)
    if source == 'Taiwan Stocks':
        symbols = [s.split()[0] for s in symbols]
    return symbols

def on_plot_prc_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        interval = interval_dropown2.value
        period = period_dropdown2.value
        if not selected:
            print("No Stock Selected!")
        elif stock_chart is mpl_mansfield.StockChart:
            stock_chart.plot(selected[0], interval=interval, period=period,
                             style=stock_theme_dropdown.value,
                             legend_loc='upper left')
        else: # Plotly
            stock_chart.plot(selected[0], interval=interval, period=period,
                             template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

import vistock.mpl.financials as mpl_fin
import vistock.plotly.financials as ply_fin

fin_chart = {
    'mplfinance': mpl_fin,
    'Plotly': ply_fin,
}[backend]

def on_plot_fin_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        elif fin_chart is mpl_fin:
            fin_chart.plot(selected[0], style=stock_theme_dropdown.value)
        else: # Plotly
            fin_chart.plot(selected[0], template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

def on_report_q_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Quarterly Financials:")
            display(yf.Ticker(ticker).quarterly_financials)
    outputs.children = (new_output,) + outputs.children

def on_report_a_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Annual Financials:")
            display(yf.Ticker(ticker).financials)
    outputs.children = (new_output,) + outputs.children

def on_clear_last_click(b):
    if outputs.children:
        children = list(outputs.children)
        children.pop(0)
        outputs.children = tuple(children)

def on_clear_all_click(b):
    outputs.children = ()

btn_plot_prc.on_click(on_plot_prc_click)
btn_plot_fin.on_click(on_plot_fin_click)
btn_report_q.on_click(on_report_q_click)
btn_report_a.on_click(on_report_a_click)
btn_clear_last.on_click(on_clear_last_click)
btn_clear_all.on_click(on_clear_all_click)

if backend == 'Plotly':
    enable_plotly_in_cell()
